In [2]:
# 独立坐标收集程序
import os
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

def background_subtract(img_array):
    """Subtracts background using median value."""
    background_value = np.median(img_array)
    subtracted = np.clip(img_array - background_value, 0, 255)
    return subtracted.astype(img_array.dtype)
    
def find_bead_center(img_array, min_area=100):
    """Finds the center of the largest circular object using contours."""
    img_blurred = cv2.medianBlur(img_array, 5)
    _, thresh = cv2.threshold(img_blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        print("Warning: No contours found.")
        return None

    valid_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_area]
    if not valid_contours:
        print(f"Warning: No contours found with area > {min_area}.")
        return None

    largest_contour = max(valid_contours, key=cv2.contourArea)

    if len(largest_contour) < 5:
        print("Warning: Largest contour has less than 5 points, using moments for centroid.")
        M = cv2.moments(largest_contour)
        if M["m00"] == 0: return None
        center_x = int(M["m10"] / M["m00"])
        center_y = int(M["m01"] / M["m00"])
        return (center_x, center_y)
    else:
        try:
            ellipse = cv2.fitEllipse(largest_contour)
            center_x, center_y = map(int, ellipse[0])
            return (center_x, center_y)
        except cv2.error as e:
            print(f"Warning: cv2.fitEllipse failed: {e}. Using moments.")
            M = cv2.moments(largest_contour)
            if M["m00"] == 0: return None
            center_x = int(M["m10"] / M["m00"])
            center_y = int(M["m01"] / M["m00"])
            return (center_x, center_y)
            
# 使用原始配置参数
base_path = r"D:\bead image\Beadimages\Beadimage1"
min_shape = (512, 512)  

# 获取所有移动微珠图像文件
files = [f for f in os.listdir(base_path) 
        if f.startswith("Moving") and f.endswith(".png")]

# 初始化坐标存储
center_coordinates = []

for fname in files:
    try:
        # 加载并预处理图像
        img_path = os.path.join(base_path, fname)
        img = Image.open(img_path).convert("L")
        img_array = np.array(img)[:min_shape[0], :min_shape[1]]
        
        # 背景减去（使用原始函数）
        img_processed = background_subtract(img_array)
        
        # 检测圆心
        center = find_bead_center(img_processed)
        
        # 记录结果
        center_coordinates.append({
            'filename': fname,
            'center_x': center[0] if center else None,
            'center_y': center[1] if center else None
        })
        
    except Exception as e:
        print(f"处理 {fname} 失败: {str(e)}")
        center_coordinates.append({
            'filename': fname,
            'center_x': None,
            'center_y': None
        })

# 输出结果
print("\n检测到的圆心坐标：")
for coord in center_coordinates[:10]:  # 打印前10个示例
    print(f"{coord['filename']}: ({coord['center_x']}, {coord['center_y']})")

# ... 前面的导入和配置保持不变 ...

# 新增输出目录
output_dir = os.path.join(base_path, "marked_images")
os.makedirs(output_dir, exist_ok=True)

for i, fname in enumerate(files):
    try:
        # 加载并预处理图像（必须每次重新加载）
        img_path = os.path.join(base_path, fname)
        img = Image.open(img_path).convert('L')
        img_array = np.array(img)[:min_shape[0], :min_shape[1]]  # 原始图像数组
        
        # 背景减去
        img_processed = background_subtract(img_array)
        
        # 检测圆心（只执行一次）
        center = find_bead_center(img_processed)
        
        # 记录结果
        center_coordinates.append({
            'filename': fname,
            'center_x': center[0] if center else None,
            'center_y': center[1] if center else None
        })

        if center:
            plt.figure(figsize=(8, 6))
            plt.imshow(img_array, cmap='gray')  # 使用原始图像数组
            cross_size = 15
            # 水平线
            plt.plot([center[0]-cross_size, center[0]+cross_size], 
                    [center[1], center[1]], 'r-', linewidth=2)
            # 垂直线
            plt.plot([center[0], center[0]],
                    [center[1]-cross_size, center[1]+cross_size], 'r-', linewidth=2)
            plt.title(f'Bead Center: {center} ({fname})')
            plt.savefig(os.path.join(output_dir, f'marked_{fname}'))
            plt.close()

    except Exception as e:

# ... 后续的结果输出和保存保持不变 ...
# 保存为DataFrame
        df_centers = pd.DataFrame(center_coordinates) # Now properly aligned
#df_centers.to_csv("detected_centers.csv", index=False)
#print("\n完整结果已保存到 detected_centers.csv")


检测到的圆心坐标：
MovingBead-0_-1000nm.png: (51, 47)
MovingBead-0_-1044nm.png: (51, 46)
MovingBead-0_-1087nm.png: (51, 46)
MovingBead-0_-1131nm.png: (51, 46)
MovingBead-0_-1171nm.png: (51, 46)
MovingBead-0_-1217nm.png: (51, 46)
MovingBead-0_-123nm.png: (51, 47)
MovingBead-0_-1263nm.png: (51, 47)
MovingBead-0_-1302nm.png: (51, 46)
MovingBead-0_-1350nm.png: (51, 47)
